In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1Y7LJkqOOg9WFj-NNfZFNdP-DexrPrk_1", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/01_00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Recursive Reasoning Fundamentals: Why Thinking Again Beats Thinking Bigger

*Part 1 of the Vizuara series on Tiny Recursive Models*
*Estimated time: 30 minutes*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/tiny-recursive-models/practice/1/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Listen: Motivation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_01_motivation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 1. Why Does This Matter?

Have you ever noticed how you solve a hard Sudoku puzzle? You do not fill in every cell in one pass. You **scan**, fill in what you can, and then **go back to the beginning** and scan again. Each pass reveals new information that was hidden in the previous pass.

This is **recursive reasoning** — repeatedly applying the same thinking process, where each application builds on the insights from the last.

Here is the stunning part: a tiny neural network with just **7 million parameters** that uses this recursive strategy outperforms trillion-parameter language models on reasoning benchmarks. GPT-4, Gemini, Claude — they all score **0%** on extreme Sudoku. The tiny recursive model scores **87.4%**.

By the end of this notebook, you will:
- Understand why recursion is so powerful for reasoning tasks
- Build a constraint-propagation Sudoku solver that demonstrates recursive reasoning
- **See** the solver progressively fill in a grid pass by pass
- Understand why single-pass approaches fundamentally fail on these tasks

Let us start by visualizing what we are going to build:

In [ ]:
#@title 🎧 Listen: Teaser Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_02_teaser_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 🎯 Teaser: By the end, you will see a solver like this
# (Don't worry about the code yet — just run this cell to see the goal)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
titles = ["Pass 1: 20% solved", "Pass 2: 50% solved", "Pass 3: 80% solved", "Pass 4: 100% solved!"]
colors_map = {0: '#ffcdd2', 1: '#c8e6c9'}  # red=unsolved, green=solved

# Simulate progressive solving
grids = [
    [[1,0,3],[0,2,0],[3,0,1]],
    [[1,2,3],[0,2,0],[3,0,1]],
    [[1,2,3],[0,2,0],[3,3,1]],
    [[1,2,3],[3,2,1],[3,1,1]],
]
for ax, grid, title in zip(axes, grids, titles):
    ax.set_xlim(0, 3); ax.set_ylim(0, 3)
    for i in range(3):
        for j in range(3):
            val = grid[i][j]
            color = '#c8e6c9' if val > 0 else '#ffcdd2'
            ax.add_patch(patches.Rectangle((j, 2-i), 1, 1, facecolor=color, edgecolor='gray'))
            if val > 0:
                ax.text(j+0.5, 2-i+0.5, str(val), ha='center', va='center', fontsize=16, fontweight='bold')
            else:
                ax.text(j+0.5, 2-i+0.5, '?', ha='center', va='center', fontsize=16, color='#999')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_aspect('equal')
plt.suptitle("Recursive Reasoning: Each Pass Reveals More", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_03_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

### Single-Pass vs Recursive Reasoning

Imagine two students taking an exam.

**Student A (Single-Pass):** Reads each question once, writes an answer immediately, and moves to the next. Never looks back. This is how most neural networks work — the input flows forward through the layers once and produces an output.

**Student B (Recursive):** Reads all the questions first, answers the easy ones, then goes back to the beginning. On the second pass, the answers from the easy questions give hints about the harder ones. On the third pass, even more becomes clear. By the fifth pass, everything falls into place.

Student B uses the **same brain** on every pass. No extra neurons were grown between passes. The only thing that changed was the **information available** — each pass created new context that made the next pass easier.

This is the fundamental insight behind recursive reasoning models:

> **You do not need a bigger brain. You need to think again.**

### Why LLMs Fail at Structured Reasoning

Large language models like GPT-4 process input through ~100 transformer layers in a single forward pass. Each layer has **unique** weights — that is hundreds of billions of parameters, each used exactly once.

But for a task like Sudoku, what matters is not how many different operations you can perform — it is how many times you can **revisit and refine** your work. A single pass, no matter how wide, cannot propagate constraints that only become visible after earlier constraints are resolved.

### 🤔 Think About This

Before we write any code, consider this puzzle:

```
| 1 | ? | 3 |
| ? | 2 | ? |
| 3 | ? | 1 |
```

Can you fill in all the missing values in a **single glance**? Or do you need to work through it step by step, where filling in one cell reveals what another cell must be?

This is the key question that motivates everything in this notebook.

In [ ]:
#@title 🎧 Listen: Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_04_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics (Light)

We will keep the math simple in this notebook — the heavy equations come in Notebooks 2 and 3.

The core idea of recursive reasoning can be expressed in two lines:

$$z \leftarrow f(x, y, z) \quad \text{(update reasoning state)}$$
$$y \leftarrow g(y, z) \quad \text{(refine solution)}$$

Where:
- $x$ is the input (the puzzle)
- $y$ is the current solution (our best guess so far)
- $z$ is the reasoning state (our mental scratchpad of constraints and deductions)
- $f$ and $g$ are the **same** function (network) applied repeatedly

Computationally, this means: take the puzzle $x$, the current guess $y$, and your notes $z$, run them through a small network to update your notes, then use the updated notes to improve your guess. Repeat $n$ times.

The beautiful thing is that $f$ and $g$ share the same weights. The model does not grow between passes — it just thinks longer.

In [ ]:
#@title 🎧 Listen: Build Puzzle
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_05_build_puzzle.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let's Build It — A Recursive Constraint Propagation Solver

We are going to build a classical constraint-propagation Sudoku solver that embodies recursive reasoning. This is not a neural network (yet!) — it is a clean algorithm that demonstrates **why** recursive passes are so powerful.

### 4.1 Setting Up the Puzzle

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy

# We will use 4x4 mini-Sudoku for clarity
# Rules: each row, column, and 2x2 box must contain {1, 2, 3, 4}

def create_puzzle():
    """Create a 4x4 Sudoku puzzle with some cells missing (0 = empty)."""
    # A valid completed 4x4 grid
    solution = np.array([
        [1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 1, 4, 3],
        [4, 3, 2, 1]
    ])
    # Remove some cells to create a puzzle
    puzzle = solution.copy()
    # Remove cells at these positions
    remove = [(0,1), (0,3), (1,0), (1,2), (2,1), (2,3), (3,0), (3,2)]
    for r, c in remove:
        puzzle[r, c] = 0
    return puzzle, solution

puzzle, solution = create_puzzle()
print("Puzzle (0 = empty):")
print(puzzle)
print("\nSolution:")
print(solution)

In [ ]:
#@title 🎧 Listen: Reasoning State
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_06_reasoning_state.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Representing the Reasoning State (z)

In the TRM paper, the reasoning state $z$ holds all the intermediate constraints the model uses to reason. In our classical solver, this corresponds to the **possibility sets** — for each empty cell, what values could it still take?

In [ ]:
def initialize_possibilities(grid):
    """
    Initialize the reasoning state z: for each cell, track which values are still possible.
    This is our 'scratchpad' — the internal reasoning that is not part of the final answer.
    """
    size = grid.shape[0]
    values = set(range(1, size + 1))
    possibilities = {}

    for r in range(size):
        for c in range(size):
            if grid[r, c] == 0:
                # Start with all values possible
                possibilities[(r, c)] = values.copy()
            else:
                # Already filled — exactly one possibility
                possibilities[(r, c)] = {grid[r, c]}

    return possibilities

z = initialize_possibilities(puzzle)
print("Initial reasoning state (possibilities for each cell):")
for (r, c), vals in sorted(z.items()):
    status = f"  ({r},{c}): {sorted(vals)}" + (" ← KNOWN" if len(vals) == 1 else "")
    print(status)

In [ ]:
#@title 🎧 Listen: Viz Possibilities
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_07_viz_possibilities.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 📊 Visualization: Show the puzzle with possibility counts
def visualize_puzzle_with_possibilities(grid, possibilities, title="", ax=None):
    """Visualize the grid with color-coded cells showing how constrained each cell is."""
    size = grid.shape[0]
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))

    for r in range(size):
        for c in range(size):
            val = grid[r, c]
            n_poss = len(possibilities.get((r, c), {val}))

            # Color by number of possibilities: fewer = more constrained = greener
            if val > 0:
                color = '#c8e6c9'  # green: solved
            elif n_poss == 1:
                color = '#a5d6a7'  # light green: deduced
            elif n_poss == 2:
                color = '#fff9c4'  # yellow: getting close
            else:
                color = '#ffcdd2'  # red: many possibilities

            ax.add_patch(patches.Rectangle((c, size-1-r), 1, 1,
                         facecolor=color, edgecolor='gray', linewidth=2))

            if val > 0:
                ax.text(c+0.5, size-1-r+0.5, str(val), ha='center', va='center',
                       fontsize=20, fontweight='bold')
            else:
                poss_str = ','.join(map(str, sorted(possibilities[(r,c)])))
                ax.text(c+0.5, size-1-r+0.65, '?', ha='center', va='center',
                       fontsize=16, color='#999')
                ax.text(c+0.5, size-1-r+0.3, f'{{{poss_str}}}', ha='center', va='center',
                       fontsize=8, color='#666')

    # Draw 2x2 box borders
    for i in range(0, size+1, 2):
        ax.axhline(y=i, color='black', linewidth=3)
        ax.axvline(x=i, color='black', linewidth=3)

    ax.set_xlim(0, size); ax.set_ylim(0, size)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=14, fontweight='bold')
    return ax

visualize_puzzle_with_possibilities(puzzle, z, "Initial State: Many Possibilities")
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: One Pass
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_08_one_pass.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 One Pass of Constraint Propagation

Now here is the core operation — one **pass** through the grid. This is the function $f$ in our recursion: it looks at the current grid and the current reasoning state, and eliminates impossible values.

In [ ]:
def get_box_idx(r, c, box_size=2):
    """Get which 2x2 box a cell belongs to."""
    return (r // box_size, c // box_size)

def one_pass(grid, possibilities):
    """
    One pass of constraint propagation.
    This is f(x, y, z) — it updates z (possibilities) based on x (puzzle) and y (current grid).

    Returns: updated grid, updated possibilities, number of cells newly solved
    """
    size = grid.shape[0]
    grid = grid.copy()
    possibilities = deepcopy(possibilities)
    newly_solved = 0

    for r in range(size):
        for c in range(size):
            if grid[r, c] != 0:
                continue  # Already solved

            # Get values already used in same row, column, and box
            row_vals = set(grid[r, :]) - {0}
            col_vals = set(grid[:, c]) - {0}

            box_r, box_c = get_box_idx(r, c)
            box = grid[box_r*2:(box_r+1)*2, box_c*2:(box_c+1)*2]
            box_vals = set(box.flatten()) - {0}

            # Eliminate impossible values from possibilities
            used = row_vals | col_vals | box_vals
            possibilities[(r, c)] -= used

            # If only one possibility remains, fill it in!
            if len(possibilities[(r, c)]) == 1:
                val = list(possibilities[(r, c)])[0]
                grid[r, c] = val
                newly_solved += 1

    return grid, possibilities, newly_solved

# Run ONE pass
grid_after_1, z_after_1, solved_1 = one_pass(puzzle, z)
print(f"After Pass 1: solved {solved_1} new cells")
print(grid_after_1)

In [ ]:
# 📊 Visualize the state after one pass
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
visualize_puzzle_with_possibilities(puzzle, z, "Before Pass 1", axes[0])
visualize_puzzle_with_possibilities(grid_after_1, z_after_1, f"After Pass 1 (+{solved_1} cells)", axes[1])
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Full Loop
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_09_full_loop.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 🤔 Think About This

After one pass, did we solve all the cells? Probably not. But here is the key insight: the cells we **did** solve now create new constraints that make previously-unsolvable cells solvable.

This is exactly why recursion is necessary — **information propagates across passes**.

### 4.4 The Full Recursive Loop

Now let us run multiple passes and watch the puzzle get solved incrementally:

In [ ]:
def recursive_solve(puzzle, max_passes=10, verbose=True):
    """
    Solve a Sudoku puzzle using recursive constraint propagation.

    This is the core recursive loop:
        z ← f(x, y, z)    (update reasoning state by eliminating impossibilities)
        y ← g(y, z)        (fill in cells that have only one possibility)
    Repeat until solved or stuck.
    """
    grid = puzzle.copy()
    possibilities = initialize_possibilities(grid)
    history = [(grid.copy(), deepcopy(possibilities), 0)]  # Track each pass

    for pass_num in range(1, max_passes + 1):
        grid, possibilities, newly_solved = one_pass(grid, possibilities)
        history.append((grid.copy(), deepcopy(possibilities), newly_solved))

        n_empty = np.sum(grid == 0)
        n_total = grid.size
        pct_solved = 100 * (n_total - n_empty) / n_total

        if verbose:
            print(f"Pass {pass_num}: solved {newly_solved} new cells | "
                  f"Total: {pct_solved:.0f}% complete | "
                  f"Remaining: {n_empty} empty cells")

        if n_empty == 0:
            if verbose:
                print(f"\n🎉 Puzzle solved in {pass_num} passes!")
            break

        if newly_solved == 0:
            if verbose:
                print(f"\n⚠️ Stuck after {pass_num} passes — need more advanced techniques")
            break

    return grid, history

solved_grid, history = recursive_solve(puzzle)

In [ ]:
#@title 🎧 Listen: Todo
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_10_todo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 📊 Visualize ALL passes side by side
n_passes = len(history)
fig, axes = plt.subplots(1, n_passes, figsize=(5 * n_passes, 5))
if n_passes == 1:
    axes = [axes]

for i, (grid_state, poss_state, n_solved) in enumerate(history):
    n_filled = np.sum(grid_state > 0)
    pct = 100 * n_filled / grid_state.size
    title = f"Pass {i}" if i > 0 else "Initial"
    title += f"\n{pct:.0f}% solved"
    if i > 0:
        title += f" (+{n_solved})"
    visualize_puzzle_with_possibilities(grid_state, poss_state, title, axes[i])

plt.suptitle("Recursive Reasoning in Action", fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 5. 🔧 Your Turn

### TODO: Implement a harder puzzle solver

Now it is your turn. The puzzle above was easy enough for basic constraint propagation. Let us try a harder one where **naked pairs** are needed — a technique where if two cells in a row both have only possibilities {a, b}, then no other cell in that row can have a or b.

In [ ]:
def create_hard_puzzle():
    """A harder 4x4 puzzle that requires naked pair elimination."""
    solution = np.array([
        [2, 4, 1, 3],
        [3, 1, 4, 2],
        [4, 3, 2, 1],
        [1, 2, 3, 4]
    ])
    puzzle = solution.copy()
    # Remove more cells
    remove = [(0,0), (0,1), (0,2), (1,1), (1,2), (1,3), (2,0), (2,2), (3,0), (3,3)]
    for r, c in remove:
        puzzle[r, c] = 0
    return puzzle, solution

hard_puzzle, hard_solution = create_hard_puzzle()
print("Hard puzzle:")
print(hard_puzzle)

In [ ]:
#@title 🎧 Listen: Experiment
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_11_experiment.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
def one_pass_with_naked_pairs(grid, possibilities):
    """
    Enhanced constraint propagation that also detects naked pairs.

    A naked pair: if two cells in the same row/col/box both have exactly
    the same two possibilities {a, b}, then a and b can be eliminated from
    all OTHER cells in that row/col/box.

    Returns: updated grid, updated possibilities, number of cells newly solved
    """
    size = grid.shape[0]
    grid = grid.copy()
    possibilities = deepcopy(possibilities)
    newly_solved = 0

    # ============ TODO ============
    # Step 1: Run basic constraint elimination (same as one_pass above)
    #         For each empty cell, eliminate values used in same row, col, box
    #
    # Step 2: Naked pair detection — for each row:
    #         - Find all cells with exactly 2 possibilities
    #         - If two cells share the same 2-element set, remove those
    #           values from all OTHER cells in the row
    #         (Repeat for columns and boxes)
    #
    # Step 3: Fill in any cells that now have exactly 1 possibility
    # ==============================

    # Step 1: Basic elimination
    for r in range(size):
        for c in range(size):
            if grid[r, c] != 0:
                continue
            row_vals = set(grid[r, :]) - {0}
            col_vals = set(grid[:, c]) - {0}
            box_r, box_c = get_box_idx(r, c)
            box = grid[box_r*2:(box_r+1)*2, box_c*2:(box_c+1)*2]
            box_vals = set(box.flatten()) - {0}
            possibilities[(r, c)] -= (row_vals | col_vals | box_vals)

    # Step 2: Naked pairs in rows
    # YOUR CODE HERE — find naked pairs and eliminate
    pass  # Replace this with your implementation

    # Step 3: Fill in solved cells
    for r in range(size):
        for c in range(size):
            if grid[r, c] == 0 and len(possibilities.get((r, c), set())) == 1:
                grid[r, c] = list(possibilities[(r, c)])[0]
                newly_solved += 1

    return grid, possibilities, newly_solved

In [ ]:
# ✅ Verification: Test with the hard puzzle
# First, try without naked pairs
grid_basic, history_basic = recursive_solve(hard_puzzle, verbose=True)
n_empty_basic = np.sum(grid_basic == 0)
print(f"\nBasic solver: {n_empty_basic} cells remaining unsolved")

# Now try YOUR enhanced solver
grid_enhanced = hard_puzzle.copy()
poss_enhanced = initialize_possibilities(grid_enhanced)
for i in range(10):
    grid_enhanced, poss_enhanced, n = one_pass_with_naked_pairs(grid_enhanced, poss_enhanced)
    if n == 0 or np.sum(grid_enhanced == 0) == 0:
        break

n_empty_enhanced = np.sum(grid_enhanced == 0)
if n_empty_enhanced < n_empty_basic:
    print(f"✅ Your enhanced solver solved more cells! ({n_empty_basic - n_empty_enhanced} additional)")
elif n_empty_enhanced == 0:
    print("✅ Perfect! Your solver completed the entire puzzle!")
else:
    print(f"❌ Enhanced solver didn't improve. {n_empty_enhanced} cells still empty. Check your naked pair logic.")

## 6. Putting It All Together — Single Pass vs Recursive

Let us now run a definitive experiment: how does a **single pass** compare to **multiple recursive passes**?

In [ ]:
#@title 🎧 Listen: Final
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_12_final.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
def single_pass_solve(puzzle):
    """Attempt to solve with exactly one pass — no recursion."""
    grid = puzzle.copy()
    possibilities = initialize_possibilities(grid)
    grid, possibilities, n_solved = one_pass(grid, possibilities)
    return grid, n_solved

def generate_random_4x4_puzzle(n_remove=8, seed=None):
    """Generate a random valid 4x4 Sudoku puzzle."""
    if seed is not None:
        np.random.seed(seed)

    # Start from a known valid grid and shuffle
    base = np.array([
        [1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 1, 4, 3],
        [4, 3, 2, 1]
    ])

    # Permute values
    perm = np.random.permutation(4) + 1
    grid = np.zeros_like(base)
    for i in range(4):
        grid[base == i+1] = perm[i]

    # Shuffle rows within bands and columns within stacks
    if np.random.random() > 0.5:
        grid[[0,1]] = grid[[1,0]]
    if np.random.random() > 0.5:
        grid[[2,3]] = grid[[3,2]]
    if np.random.random() > 0.5:
        grid[:,[0,1]] = grid[:,[1,0]]
    if np.random.random() > 0.5:
        grid[:,[2,3]] = grid[:,[3,2]]

    solution = grid.copy()
    puzzle = grid.copy()

    # Remove cells
    indices = np.random.choice(16, size=min(n_remove, 16), replace=False)
    for idx in indices:
        puzzle[idx // 4, idx % 4] = 0

    return puzzle, solution

# Run experiment on many puzzles
np.random.seed(42)
n_puzzles = 100
single_pass_scores = []
recursive_scores = []

for i in range(n_puzzles):
    puz, sol = generate_random_4x4_puzzle(n_remove=8, seed=42+i)

    # Single pass
    grid_1, _ = single_pass_solve(puz)
    score_1 = np.sum(grid_1 == sol) / sol.size
    single_pass_scores.append(score_1)

    # Recursive (up to 10 passes)
    grid_r, _ = recursive_solve(puz, max_passes=10, verbose=False)
    score_r = np.sum(grid_r == sol) / sol.size
    recursive_scores.append(score_r)

print(f"Single pass — Average accuracy: {np.mean(single_pass_scores)*100:.1f}%")
print(f"Recursive  — Average accuracy: {np.mean(recursive_scores)*100:.1f}%")
print(f"\nImprovement from recursion: +{(np.mean(recursive_scores) - np.mean(single_pass_scores))*100:.1f} percentage points")

In [ ]:
# 📊 Visualize the comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
methods = ['Single Pass', 'Recursive\n(up to 10 passes)']
scores = [np.mean(single_pass_scores)*100, np.mean(recursive_scores)*100]
colors = ['#ffcdd2', '#c8e6c9']
bars = axes[0].bar(methods, scores, color=colors, edgecolor='gray', linewidth=2)
axes[0].set_ylabel('Accuracy (%)', fontsize=12)
axes[0].set_title('Single Pass vs Recursive Solving', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 105)
for bar, score in zip(bars, scores):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                f'{score:.1f}%', ha='center', fontsize=14, fontweight='bold')

# Histogram of scores
axes[1].hist(np.array(single_pass_scores)*100, bins=10, alpha=0.7, color='#ef9a9a', label='Single Pass', edgecolor='gray')
axes[1].hist(np.array(recursive_scores)*100, bins=10, alpha=0.7, color='#a5d6a7', label='Recursive', edgecolor='gray')
axes[1].set_xlabel('Accuracy (%)', fontsize=12)
axes[1].set_ylabel('Number of Puzzles', fontsize=12)
axes[1].set_title('Score Distribution (100 puzzles)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=12)

plt.tight_layout()
plt.show()

## 7. 🎯 Final Output — Animated Recursive Solving

Let us create a beautiful visualization that shows the recursive solver working through a puzzle step by step.

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_13_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
def animate_solving(puzzle, solution):
    """Create a panel visualization showing the puzzle being solved pass by pass."""
    grid = puzzle.copy()
    possibilities = initialize_possibilities(grid)

    states = [(grid.copy(), deepcopy(possibilities))]
    for _ in range(10):
        grid, possibilities, n = one_pass(grid, possibilities)
        states.append((grid.copy(), deepcopy(possibilities)))
        if np.sum(grid == 0) == 0 or n == 0:
            break

    n_states = len(states)
    fig, axes = plt.subplots(1, n_states, figsize=(4.5 * n_states, 5))
    if n_states == 1:
        axes = [axes]

    size = puzzle.shape[0]
    for idx, (g, p) in enumerate(states):
        ax = axes[idx]
        n_filled = np.sum(g > 0)
        n_total = g.size
        n_original = np.sum(puzzle > 0)

        for r in range(size):
            for c in range(size):
                val = g[r, c]

                if puzzle[r, c] > 0:
                    color = '#e3f2fd'  # blue: given
                    fontcolor = '#1565c0'
                elif val > 0:
                    color = '#c8e6c9'  # green: solved by algorithm
                    fontcolor = '#2e7d32'
                else:
                    n_poss = len(p.get((r, c), set()))
                    color = '#fff9c4' if n_poss <= 2 else '#ffcdd2'
                    fontcolor = '#999'

                ax.add_patch(patches.Rectangle((c, size-1-r), 1, 1,
                             facecolor=color, edgecolor='gray', linewidth=2))

                if val > 0:
                    ax.text(c+0.5, size-1-r+0.5, str(val), ha='center', va='center',
                           fontsize=18, fontweight='bold', color=fontcolor)
                else:
                    ax.text(c+0.5, size-1-r+0.5, '?', ha='center', va='center',
                           fontsize=16, color='#bbb')

        # Draw 2x2 box borders
        for i in range(0, size+1, 2):
            ax.axhline(y=i, color='black', linewidth=3)
            ax.axvline(x=i, color='black', linewidth=3)

        ax.set_xlim(0, size); ax.set_ylim(0, size)
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_aspect('equal')

        pct = 100 * n_filled / n_total
        label = "Initial" if idx == 0 else f"Pass {idx}"
        ax.set_title(f"{label}\n{pct:.0f}% filled", fontsize=13, fontweight='bold')

    plt.suptitle("🚀 Recursive Constraint Propagation in Action",
                 fontsize=16, fontweight='bold', y=1.02)

    # Legend
    fig.text(0.15, -0.02, "🔵 Given", fontsize=11, color='#1565c0')
    fig.text(0.35, -0.02, "🟢 Solved by recursion", fontsize=11, color='#2e7d32')
    fig.text(0.6, -0.02, "🔴 Still unknown", fontsize=11, color='#c62828')

    plt.tight_layout()
    plt.show()

    if np.sum(g == 0) == 0:
        print("🎉 Puzzle solved!")
        print(f"   Required {len(states)-1} recursive passes")
        print(f"   Same constraint rules applied each time — only the information changed")
    else:
        print(f"⚠️ Could not fully solve — {np.sum(g == 0)} cells remaining")

# Generate a fresh puzzle and solve it
puz, sol = generate_random_4x4_puzzle(n_remove=9, seed=123)
animate_solving(puz, sol)

In [ ]:
# Let us do a few more!
print("=" * 60)
print("  Three more puzzles solved recursively")
print("=" * 60)

for seed in [7, 99, 2025]:
    print(f"\n--- Puzzle (seed={seed}) ---")
    puz, sol = generate_random_4x4_puzzle(n_remove=8, seed=seed)
    animate_solving(puz, sol)

## 8. Reflection and Next Steps

### 💡 Key Takeaways

1. **Recursive reasoning works** because each pass creates new information that enables the next pass — constraints propagate across iterations
2. **The same function is applied each time** — no new parameters or logic between passes, just new context from previous results
3. **Single-pass approaches fail** on structured reasoning tasks because they cannot propagate long-range dependencies that only emerge after intermediate steps are resolved

### 🤔 Reflection Questions

1. What would happen if the puzzle required **backtracking** (guessing and checking)? Could pure constraint propagation handle that, or would we need a fundamentally different approach?
2. How is constraint propagation similar to what happens inside a recursive neural network? What does the **reasoning state z** correspond to in our solver?
3. Why do you think large language models score **0%** on extreme Sudoku despite having trillions of parameters?

### 🏆 Optional Challenges

1. **Scale up:** Implement constraint propagation for a full 9×9 Sudoku. How many passes does it typically need?
2. **Backtracking:** Add a guessing mechanism — when stuck, pick the cell with fewest possibilities, guess a value, and continue. If it leads to a contradiction, backtrack. This is closer to how the neural model handles uncertainty.
3. **Visualize z:** Create a heatmap that shows how the "uncertainty" (number of possibilities per cell) decreases with each pass. This is a direct analog of how the latent reasoning state z evolves in TRM.

### What's Next

In the next notebook, we will build the actual **Tiny Recursive Model** neural network from scratch — replacing our hand-coded constraint propagation with a learned function that discovers its own reasoning strategy.